In [63]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text

In [64]:

# Configuración de la conexión a PostgreSQL
DB_CONFIG = {
    'dbname': 'DATAWAREHOUSE_ESSI',
    'user': 'postgres',
    'password': 'Password2',
    'host': '10.0.1.6',
    'port': '5432',
}

TABLAS_ATENCION = [
    "dw_consulta_externa",
    "dw_emergencia_egresos",
    "dw_hospitalizacion_egresos",
    "dw_centro_quirurgico",
]

LISTA_UNICA = "mtd_lista_unica_pacientes"

TIPO_BUSQUEDA = 10

DW_CONSOLIDADO = "consolidado_atenciones_ryh"

# Mapeo de nombres de columna de fecha según la tabla de atención
MAPEO_FECHA_ATENCION = {
    "dw_consulta_externa": "fecha_atencion",
    "dw_emergencia_egresos": "fec_altadm",
    "dw_hospitalizacion_egresos": "fec_egreso",
    "dw_centro_quirurgico": "fec_oper"
}

# Columnas estándar en las tablas temporales
COLUMNAS_STD = [
    "cod_oricentro", "cod_centro", "acto_med", "periodo", "fecha_atencion", 
    "cmame_pacsecnum", "cod_tipdoc_paciente", "doc_paciente", "anio_edad", "sexo",
    "cod_conddiag", "cod_diagnostico", "cod_orden", "cod_tipodiag", "area", 
    "anio_busqueda", "cod_secuencia", "cas_adscripcion", "cod_servicio", "cod_actividad",
    "cod_subactividad", "cod_cartera", "cod_cpms", "cod_tipdoc_medico", "dni_medico", "num_solicitud"
]

In [65]:
engine = create_engine(f'postgresql://{DB_CONFIG["user"]}:{DB_CONFIG["password"]}@{DB_CONFIG["host"]}:{DB_CONFIG["port"]}/{DB_CONFIG["dbname"]}')
connection = engine.connect()

In [66]:
# Crear tabla temporal si no existe
connection.execute(text(f"""
    CREATE TABLE IF NOT EXISTS public.test2 (
        cod_oricentro VARCHAR(1),
        cod_centro VARCHAR(3),
        acto_med NUMERIC(10),
        periodo VARCHAR(6),
        fecha_atencion VARCHAR(10),  -- Se mantiene como string "DD/MM/YYYY"
        cmame_pacsecnum VARCHAR(20),
        cod_tipdoc_paciente VARCHAR(2),
        doc_paciente VARCHAR(20),
        anio_edad INT,
        sexo CHAR(1),
        area VARCHAR(500),
        anio_busqueda VARCHAR(500),
        cas_adscripcion VARCHAR(500),
        cod_servicio VARCHAR(20),
        cod_actividad VARCHAR(30),
        cod_subactividad VARCHAR(30),
        cod_cartera VARCHAR(30),
        cod_cpms VARCHAR(30),
        cod_tipdoc_medico VARCHAR(30),
        dni_medico VARCHAR(30),
        num_solicitud INT
    )
"""))

# Vaciar la tabla temporal
connection.execute(text(f"TRUNCATE TABLE {DW_CONSOLIDADO}"))

In [67]:
query_base1 = f"""select * from {DW_CONSOLIDADO} limit 10"""

base1 = pd.read_sql(query_base1, connection)

In [68]:
base1

,cod_oricentro,cod_centro,acto_med,periodo,fecha_atencion,cmame_pacsecnum,cod_tipdoc_paciente,doc_paciente,anio_edad,sexo,...,cod_secuencia,cas_adscripcion,cod_servicio,cod_actividad,cod_subactividad,cod_cartera,cod_cpms,cod_tipdoc_medico,dni_medico,num_solicitud


In [69]:
DNIS = f"""SELECT cod_tipdoc_paciente, doc_paciente FROM {LISTA_UNICA} where tipo_busqueda = {TIPO_BUSQUEDA}"""
df_lista_unica = pd.read_sql(DNIS, connection)

In [70]:
df_lista_unica = df_lista_unica.drop_duplicates(subset=['cod_tipdoc_paciente', 'doc_paciente'])

In [71]:
query = 'select * from dw_consulta_externa_2019_01'
dw = pd.read_sql(query, connection)

In [72]:
dw.shape

(400458, 49)

In [73]:
df_main = dw.merge(df_lista_unica, on=['cod_tipdoc_paciente', 'doc_paciente'], how='inner')


In [74]:
df_main.shape

(82184, 49)

In [75]:
def procesar_datos(fecha_inicio, fecha_fin):                               
        for i in range(len(TABLAS_ATENCION)):
                        tabla_a = TABLAS_ATENCION[i]
                        columna_fecha = MAPEO_FECHA_ATENCION[tabla_a]
                        for anio in range(int(fecha_inicio[:4]), int(fecha_fin[:4]) + 1):
                                for mes in range(1, 13):
                                        mes_str = f"{mes:02d}"
                                        if f"{anio}{mes_str}" < fecha_inicio or f"{anio}{mes_str}" > fecha_fin:
                                                continue                                
                                        tabla_particionada = f"{tabla_a}_{anio}_{mes_str}"
                                        print(f"Procesando tabla {tabla_particionada}...")
                                        area = "'CEXT'" if tabla_a == "dw_consulta_externa" else \
                                                "'EMER'" if tabla_a == "dw_emergencia_egresos" else \
                                                "'HOSP'" if tabla_a == "dw_hospitalizacion_egresos" else \
                                                "'CQX'" if tabla_a == "dw_centro_quirurgico" else ''
                                        cod_actividad = "cod_actividad" if tabla_a == "dw_consulta_externa" else \
                                                "''" if tabla_a == "dw_emergencia_egresos" else \
                                                "cod_actividad" if tabla_a == "dw_hospitalizacion_egresos" else \
                                                "''" if tabla_a == "dw_centro_quirurgico" else ''
                                        cod_subactividad = "cod_subactividad" if tabla_a == "dw_consulta_externa" else \
                                                "''" if tabla_a == "dw_emergencia_egresos" else \
                                                "''" if tabla_a == "dw_hospitalizacion_egresos" else \
                                                "''" if tabla_a == "dw_centro_quirurgico" else ''
                                        cod_cartera = "cod_cartera" if tabla_a == "dw_consulta_externa" else \
                                                "''" if tabla_a == "dw_emergencia_egresos" else \
                                                "''" if tabla_a == "dw_hospitalizacion_egresos" else \
                                                "''" if tabla_a == "dw_centro_quirurgico" else ''
                                        cod_cpms = "cod_cpms" if tabla_a == "dw_consulta_externa" else \
                                                "''" if tabla_a == "dw_emergencia_egresos" else \
                                                "''" if tabla_a == "dw_hospitalizacion_egresos" else \
                                                "cod_cpms" if tabla_a == "dw_centro_quirurgico" else ''
                                        cod_tipdoc_medico = "cod_tipdoc_medico" if tabla_a == "dw_consulta_externa" else \
                                                "''" if tabla_a == "dw_emergencia_egresos" else \
                                                "cod_tipdoc_medico" if tabla_a == "dw_hospitalizacion_egresos" else \
                                                "cod_tipdoc_medico" if tabla_a == "dw_centro_quirurgico" else ''
                                        dni_medico = "dni_medico" if tabla_a == "dw_consulta_externa" else \
                                                "''" if tabla_a == "dw_emergencia_egresos" else \
                                                "dni_medico" if tabla_a == "dw_hospitalizacion_egresos" else \
                                                "dni_medico" if tabla_a == "dw_centro_quirurgico" else ''
                                        num_solicitud = "num_solicitud" if tabla_a == "dw_centro_quirurgico" else 0
                                        query = f"""
                                                SELECT a.cod_oricentro, a.cod_centro, a.acto_med, a.periodo,
                                                a.{columna_fecha} AS fecha_atencion,  -- 🔹 Se mapea a "fecha_atencion"
                                                a.cmame_pacsecnum, a.cod_tipdoc_paciente, 
                                                a.doc_paciente, a.anio_edad, a.sexo,
                                                {area} AS area, a.anio as anio_busqueda,
                                                cas_adscripcion, a.cod_servicio, {cod_actividad} AS cod_actividad, 
                                                {cod_subactividad} AS cod_subactividad, {cod_cartera} AS cod_cartera,
                                                {cod_cpms} AS cod_cpms, {cod_tipdoc_medico} AS cod_tipdoc_medico, {dni_medico} AS dni_medico,
                                                {num_solicitud} AS num_solicitud
                                                FROM {tabla_particionada} a"""
                                        df_atenciones = pd.read_sql(query, connection)
                                        
                                        df_main = df_atenciones.merge(df_lista_unica, on=['cod_tipdoc_paciente', 'doc_paciente'], how='inner')

                                        print(df_main.shape)
                                        
                                        # query_base1 = f"""select * from {DW_CONSOLIDADO} limit 10"""

                                        # base1 = pd.read_sql(query_base1, connection)
                                        
                                        # comunes = set(base1.columns).intersection(set(df_main.columns)) 
                                        # base = df_main[list(comunes)]
                                        

                                        df_main.to_sql(name=f'public.test2', con=connection, if_exists='append', index=False,chunksize=50000)
                                        print("Proceso completado.")


In [76]:
procesar_datos("201901", "201901")

Procesando tabla dw_consulta_externa_2019_01...
(82184, 21)
Proceso completado.
Procesando tabla dw_emergencia_egresos_2019_01...
(8381, 21)
Proceso completado.
Procesando tabla dw_hospitalizacion_egresos_2019_01...
(329, 21)
Proceso completado.
Procesando tabla dw_centro_quirurgico_2019_01...
(233, 21)
Proceso completado.


In [77]:
# engine.dispose()

# connection.close()